# lembre-se de substituir /caminho/ pelo caminho do seu diretorio

In [ ]:
'''
Usaremos os endereços extraidos dos sites para filtrar por região da cidade. Os endereços extraidos não seguem um padrão
e alguns não apresentam os bairros, visto que os endereços no site era informado pelo usuário.
Dessa forma usaremos o geocoders para obter o bairro a partir do nome da rua e utilizar esses bairros para fazer uma 
filtragem.

OBS: o geocoders tem uma baixa especificidade, não levando em consideração o número da rua na sua busca. 
Dessa forma para algumas ruas que passam por mais 
de um bairro. Pode ocorrer que o bairro apontado não esteja 100% correto, mas já é uma forma de filtragem, 
precisando levar em consideração bairros vizinhos
'''

# codigo para bairro começa aqui

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as srn
import geopy
import re
import time
import requests
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="mygeocode")

In [ ]:
#carrega arquivos selenium

cabecalho = ['nome', 'endereco', 'crm', 'site']
caminho_home = "/home/caminho"
cainho_arquivo_dfs = caminho_home + "/primeira_etapa_dados_medicos/dados_medicos_selenium.csv"
dfs = pd.read_csv(cainho_arquivo_dfs, sep = ";", header=None)
dfs.columns = cabecalho
dfs.shape

In [ ]:

dfs.drop_duplicates(keep='first',inplace=True)
dfs.dropna(subset="endereco", inplace=True)

dfs.reset_index(drop=True,inplace =True)
dfs.shape

In [ ]:
def achar_rua(nome_rua_buscar):

    
    padrao = re.compile(r'^(?P<rua>.*?)\s*(?P<numero>\d+)')
    correspondencia = padrao.search(nome_rua_buscar)
    if correspondencia is not None:
        resultado = f'{correspondencia[0]}'
    else:
        resultado = f'{nome_rua_buscar}'

    return(resultado)

def usar_geolocalizador(nome_da_rua):
    local = geolocator.geocode(nome_da_rua)
     
    if local is not None:
        retorno = local.address
        
    else:
        retorno = "None"
    
    return(retorno)




In [ ]:
"""
como o geolocator demanda muito tempo, é preferivel salvar os dados para prosseguir
"""
cainho_arquivo_datas = caminho_home + "/segunda-etapa_analise_dados_medicos/dados_medicos_geolocator.csv"
arquivo_datas = open(cainho_arquivo_datas, 'a+')
arquivo_datas.write('nome;end_extraido_site;crm;site;end_novo')
arquivo_datas.write('\n')
tabela_formatada = []
a=0
passos = 0
total = len(dfs.endereco)
atual = 0
for i in range(len(dfs.endereco)):
    time.sleep(2)
    auxiliar = []
    auxiliar.append(dfs.iloc[i,0])
    auxiliar.append(dfs.iloc[i,1])
    auxiliar.append(dfs.iloc[i,2])
    auxiliar.append(dfs.iloc[i,3])
    nome_rua_geo = achar_rua(str(dfs.iloc[i,1]))
    if ( "rua" not in nome_rua_geo.lower() ) and ("avenida" not in nome_rua_geo.lower()):
        nome_rua_geo = f'rua {nome_rua_geo}'
    nome_rua_geo = f'{nome_rua_geo}, Minas Gerais, Belo Horizonte'
    local_geo = geolocator.geocode(nome_rua_geo, timeout=10)
    time.sleep(2)
    print (f'------------------pesquisado {nome_rua_geo}')
    print(local_geo)

    if local_geo is not None:
        auxiliar.append(str(local_geo))
    else:
        auxiliar.append("None")

    tabela_formatada.append(auxiliar)
    time.sleep(1)
    atual = atual + 1
    passos = passos +1
    salvar_esses_dados = f'{auxiliar[0]};{auxiliar[1]};{auxiliar[2]};{auxiliar[3]};{auxiliar[4]}'
    print (f'------------------dados salvos')
    print(salvar_esses_dados)
    print(f'final {atual} de {total}')
    arquivo_datas.write(salvar_esses_dados)
    arquivo_datas.write('\n')
    if passos > 20 :
        arquivo_datas.close()
        arquivo_datas = open(cainho_arquivo_datas, 'a+')
        passos = 0
arquivo_datas.close()

# Codigo para achar bairro

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as srn
import geopy
import re
import time
import copy

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="mygeocode")

In [ ]:

dfa = pd.read_csv(cainho_arquivo_datas, sep = ";")
cabecalho = ['nome', 'end_extraido_site', 'crm', 'site', 'end_novo']
dfa.columns = cabecalho
dfa['bairros'] = [None] * len(dfa)
dfa.dropna(subset="end_novo", inplace=True)
dfa.reset_index(drop=True,inplace =True)

In [ ]:

for i in range(len(dfa.end_novo)):
    bairro = None
    
    endereco_fragmentado = dfa.iloc[i,4].split(', ')
    for j in range(len(endereco_fragmentado)):
        if "rua" in endereco_fragmentado[j].lower() or "avenida" in endereco_fragmentado[j].lower() or "alameda" in endereco_fragmentado[j].lower() or "praça" in endereco_fragmentado[j].lower():
            dfa.iloc[i,5] = copy.deepcopy(endereco_fragmentado[j+1])
dfa  

In [ ]:
cainho_arquivo_bairros = caminho_home + "/segunda-etapa_analise_dados_medicos/dados_medicos_geolocator_bairros.csv"
arquivo_datas = open(cainho_arquivo_bairros, 'w+')
arquivo_datas.write("nome;end_extraido_site;crm;site;end_novo;bairros")
arquivo_datas.write('\n')
for i in range(len(dfa.end_novo)):
    escrever = f'{dfa.iloc[i][0]};{dfa.iloc[i][1]};{dfa.iloc[i][2]};{dfa.iloc[i][3]};{dfa.iloc[i][4]};{dfa.iloc[i][5]}'
    arquivo_datas.write(escrever)
    arquivo_datas.write('\n')
arquivo_datas.close()